In [125]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import RandomizedPCA
import numpy as np

In [41]:
df = pd.read_csv("UCM-BVLC_fc7.csv")

In [42]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,forest98.jpg,-0.000000,-0.0,1.020060,-0.0,3.446866,1.014767,-0.0,0.669705,-0.000000,...,-0.000000,-0.000000,1.939116,0.084468,-0.0,-0.000000,2.680483,-0.0,-0.000000,1.228164
1,freeway68.jpg,-0.000000,-0.0,-0.000000,-0.0,0.908102,-0.000000,-0.0,-0.000000,0.878464,...,-0.000000,-0.000000,2.004626,-0.000000,-0.0,-0.000000,-0.000000,-0.0,0.889653,2.823646
2,baseballdiamond62.jpg,0.910761,-0.0,1.088903,-0.0,-0.000000,-0.000000,-0.0,1.522825,-0.000000,...,0.279758,-0.000000,3.392699,1.523453,-0.0,1.085757,-0.000000,-0.0,-0.000000,-0.000000
3,chaparral68.jpg,-0.000000,-0.0,-0.000000,-0.0,-0.000000,0.628042,-0.0,-0.000000,7.859660,...,-0.000000,1.667751,-0.000000,-0.000000,-0.0,-0.000000,3.221682,-0.0,-0.000000,0.798871
4,river58.jpg,-0.000000,-0.0,0.981181,-0.0,-0.000000,-0.000000,-0.0,-0.000000,0.464536,...,-0.000000,-0.000000,-0.000000,5.334822,-0.0,-0.000000,-0.000000,-0.0,-0.000000,0.205087


In [43]:
X = df.iloc[:,1:]
print X.shape

(2100, 4096)


In [48]:
pca = RandomizedPCA(n_components=800)

res = pca.fit_transform(X) 

print sum(pca.explained_variance_ratio_)

0.944303573159


In [100]:
km = KMeans(n_clusters=30, init='k-means++',n_init=100, max_iter=500, random_state=0, n_jobs=-1)
km.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=500, n_clusters=30, n_init=100,
    n_jobs=-1, precompute_distances='auto', random_state=0, tol=0.0001,
    verbose=0)

In [101]:
clusters = list(km.labels_)

In [102]:
cluster_images = {}
for i, cluster in enumerate(clusters):
    if cluster in cluster_images.keys():
        cluster_images[cluster].append(df.iloc[i,0])
    else:
        cluster_images[cluster] = [df.iloc[i,0]]

In [122]:
src_path = 'UCMerced/JPGs/'
image_type = (".jpg", ".png", ".JPG", ".PNG", ".JPEG", ".tif", ".tiff", ".TIFF", '.TIF')
 
image_paths = []  
for root, dirs, files in os.walk(src_path):
    image_paths.extend([os.path.join(root, f) for f in files if f.endswith(image_type)])
    
print 'number of images is', len(image_paths)

number of images is 2100


In [123]:
cluster_paths = {}
for key in cluster_images.keys():
    if key in cluster_paths.keys():
        cluster_paths[key].append([image_path for image_path in image_paths if image_path.split("/")[-1] in cluster_images[key]])
    else:
        cluster_paths[key] = [image_path for image_path in image_paths if image_path.split("/")[-1] in cluster_images[key]]

In [124]:
cluster_paths[1]

['UCMerced/JPGs/intersection11.jpg',
 'UCMerced/JPGs/buildings48.jpg',
 'UCMerced/JPGs/buildings01.jpg',
 'UCMerced/JPGs/denseresidential45.jpg',
 'UCMerced/JPGs/buildings06.jpg',
 'UCMerced/JPGs/buildings08.jpg',
 'UCMerced/JPGs/freeway13.jpg',
 'UCMerced/JPGs/denseresidential21.jpg',
 'UCMerced/JPGs/denseresidential26.jpg',
 'UCMerced/JPGs/buildings19.jpg',
 'UCMerced/JPGs/denseresidential28.jpg',
 'UCMerced/JPGs/buildings17.jpg',
 'UCMerced/JPGs/buildings10.jpg',
 'UCMerced/JPGs/intersection09.jpg',
 'UCMerced/JPGs/buildings22.jpg',
 'UCMerced/JPGs/intersection07.jpg',
 'UCMerced/JPGs/denseresidential14.jpg',
 'UCMerced/JPGs/intersection00.jpg',
 'UCMerced/JPGs/intersection59.jpg',
 'UCMerced/JPGs/buildings09.jpg',
 'UCMerced/JPGs/buildings07.jpg',
 'UCMerced/JPGs/buildings00.jpg',
 'UCMerced/JPGs/buildings47.jpg',
 'UCMerced/JPGs/buildings96.jpg',
 'UCMerced/JPGs/denseresidential15.jpg',
 'UCMerced/JPGs/intersection01.jpg',
 'UCMerced/JPGs/buildings87.jpg',
 'UCMerced/JPGs/intersec

In [126]:
def copyFromListToPath(src_paths, dest_path):
    for i in range(len(src_paths)): shutil.copy2(src_paths[i], dest_path)

In [129]:
for key in cluster_paths.keys():
    dest_dir = 'clusters/' + str(key) + '/'
    if not os.path.exists(dest_dir): 
        os.makedirs(dest_dir)
    copyFromListToPath(cluster_paths[key], dest_dir)